##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [ ]:
!pip install -q -U google-generativeai

In [1]:
!pip install -q wikipedia

  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [2]:
import json
import textwrap

import google.generativeai as genai

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-1.5-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [4]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-1.5-flash')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-1.5-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [5]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Llama (language model)', 'Stochastic parrot']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Llama (language model)"
Information Source: https://en.wikipedia.org/wiki/Llama_(language_model)
Llama (language model) did not match with any page id, hence skipping.
Fetching page: "Stochastic parrot"
Information Source: https://en.wikipedia.org/wiki/Stochastic_parrot
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Llama_(language_model)
     https://en.wikipedia.org/wiki/Stochastic_parrot


Here is what the search results look like:

In [6]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> ## Relevant information about LLMs:
> 
> **Definition:**
> 
> * A large language model (LLM) is a computational model capable of general-purpose language generation and other natural language processing tasks like classification.
> * LLMs learn statistical relationships from vast amounts of text data through a self-supervised and semi-supervised training process.
> * LLMs are artificial neural networks built on the transformer architecture.
> 
> **Key Abilities:**
> 
> * **Text generation:** LLMs can generate text by predicting the next token or word in a sequence.
> * **Natural Language Processing (NLP):** LLMs can be used for tasks like classification, translation, and summarization.
> 
> **Training and Architecture:**
> 
> * **Transformer architecture:** LLMs are based on the transformer architecture, which enables efficient processing and generation of large-scale text data.
> * **Decoder-only architecture:** Most capable LLMs use a decoder-only transformer architecture.
> * **Prompt engineering:** LLMs can be guided to perform specific tasks by crafting specific input prompts.
> * **Reinforcement Learning from Human Feedback (RLHF):** LLMs can be fine-tuned using RLHF algorithms based on human preferences.
> * **Instruction tuning:** LLMs can learn to perform tasks based on human-generated corrections.
> * **Mixture of Experts (MoE):** MoE allows for training larger and more complex models.
> * **Context Window:** LLMs have a limited context window, which affects the amount of information they can consider when generating responses.
> * **Attention Mechanism:** LLMs use an attention mechanism to determine the relevance of different tokens within a context.
> 
> **Notable LLMs:**
> 
> * **OpenAI:** GPT series (GPT-3.5, GPT-4), used in ChatGPT and Microsoft Copilot.
> * **Google:** Gemini, used in the Gemini chatbot.
> * **Meta:** LLaMA family of models.
> * **Anthropic:** Claude models.
> * **Mistral AI:** Mistral models.
> 
> **History:**
> 
> * Early language models were based on statistical methods and n-gram models.
> * Neural networks became dominant in language modeling after 2012.
> * The transformer architecture was introduced in 2017, revolutionizing language modeling.
> * GPT-2 (2019) and GPT-3 (2020) gained widespread attention.
> * ChatGPT (2022) brought LLMs to the general public.
> * GPT-4 (2023) showcased increased accuracy and multimodal capabilities.
> 
> **Other Important Aspects:**
> 
> * **Dataset preprocessing:** LLMs require text data to be converted into numbers through tokenization.
> * **Training cost:** Training LLMs is computationally expensive, requiring substantial resources.
> * **Tool use:** LLMs can be integrated with external tools and APIs to perform more complex tasks.
> * **Agency:** LLMs can be used as components of intelligent agents.
> * **Compression:** Techniques like quantization are used to reduce the size of LLMs.
> * **Multimodality:** LLMs are being developed to process multiple input modalities, such as text, images, and video.
> * **Scaling laws:** Performance of LLMs is related to their size, training data size, and computational cost.
> * **Emergent abilities:** LLMs exhibit emergent abilities that arise from the complex interactions of their components.
> * **Interpretation:** Understanding how LLMs work is an active area of research.
> * **Evaluation:** LLMs are evaluated using various metrics, including perplexity, task-specific datasets, and adversarial evaluations.
> * **Wider impact:** LLMs have the potential to significantly impact various industries and aspects of society.
> 
> This information provides a comprehensive overview of LLMs, highlighting their capabilities, training process, history, and potential impact. 
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Large_language_model

> ## Relevant Information about LLMs:
> 
> **Definition:**
> 
> * **LLMs (Large Language Models)** are machine learning models that can generate human-like text.
> * The term "stochastic parrot" describes the theory that LLMs lack understanding of the meaning of the language they process. 
> * LLMs are essentially probabilistic word and sentence linkers, trained on large datasets. 
> 
> **Concerns and Criticisms:**
> 
> * LLMs are limited by the data they are trained on and can only repeat what they have learned. 
> * They are unable to discern correct or appropriate information, leading to the possibility of generating "dangerously wrong" outputs. 
> * They present dangers such as environmental and financial costs, hidden biases, potential for deception, and lack of understanding of underlying concepts.
> 
> **Debate:**
> 
> * Some argue that LLMs are merely "stochastic parrots" incapable of understanding language.
> * Others believe that with enough data, LLMs can truly understand language.
> 
> **Arguments for "Stochastic Parrot" Theory:**
> 
> * LLMs often generate "hallucinations" - fake information presented as fact.
> * They fail to understand complex or ambiguous grammar, indicating a lack of deeper meaning comprehension. 
> 
> **Arguments against "Stochastic Parrot" Theory:**
> 
> * LLMs perform well on benchmarks for reasoning, common sense, and language understanding.
> * They demonstrate an ability to predict future events based on information in prompts.
> *  "Mechanistic interpretability" studies suggest LLMs develop internal representations of the world, indicating a form of understanding. 
> 
> **Ongoing Research:**
> 
> * The debate about LLMs and understanding continues with ongoing research and analysis. 
> * While some argue that LLMs are nothing more than pattern matchers, others explore evidence suggesting potential for genuine language understanding. 
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Stochastic_parrot

### Pass the tools to the model

If you pass a list of functions to the `GenerativeModel`'s `tools` argument,
it will extract a schema from the function's signature and type hints, and then pass schema along to the API calls. In response the model may return a `FunctionCall` object asking to call the function.

Note: This approach only handles annotations of `AllowedTypes = int | float | str | dict | list['AllowedTypes']`

The `GenerativeModel` will keep a reference to the function inself, so that it _can_ execute the function locally later.

In [7]:
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [8]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [9]:
model = genai.GenerativeModel(
    'gemini-1.5-flash', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "How do deep-sea creatures survive?"
Related search terms: ['Deep-sea community', 'Still Wakes the Deep', 'Deep sea']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Still Wakes the Deep"
Information Source: https://en.wikipedia.org/wiki/Still_Wakes_the_Deep
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Searching for "What are the adaptations of deep-sea life?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea community']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Searching for "Deep-sea life survival mechanisms"
Related search terms: ['Deep-sea community', 'Deep sea mining', 'Deepsea Challenger']
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Fetching page: "Deepsea Challenger"
Information Source: https://en.wikipedia.org/wiki/Deepsea_Challenger
S

In [10]:
to_markdown(res.text)

> Deep-sea life faces unique challenges due to the extreme conditions in the ocean depths. To survive, these organisms have evolved remarkable adaptations. 
> 
> Here's a summary of how they cope:
> 
> * **Darkness:** Most deep-sea creatures live in perpetual darkness. Many have evolved bioluminescence, producing their own light for communication, hunting, and attracting mates.
> * **High Pressure:** The immense pressure at depth can crush most organisms. Deep-sea creatures have adapted with gelatinous flesh, minimal skeletal structures, and specialized proteins that resist pressure.
> * **Cold Temperatures:** The deep sea is very cold. Organisms have adapted with enzymes that function at low temperatures, and some produce antifreeze proteins to prevent ice formation.
> * **Limited Food:** Food is scarce in the deep sea. Creatures have slow metabolisms, efficient digestion, and some rely on scavenging or filter feeding.
> * **Chemosynthesis:** In areas like hydrothermal vents and cold seeps, some organisms use chemosynthesis, a process that uses chemicals from the environment to produce energy. This forms the base of unique food webs.
> 
> Deep-sea life is a testament to the incredible adaptability of life on Earth. These organisms have overcome extreme challenges to thrive in one of the most inhospitable environments on our planet. 


Check for additional citations:

In [11]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [12]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

In the chat history you can see all 4 steps:

1. The user sent the query.
2. The model replied with a `genai.protos.FunctionCall` calling the `wikipedia_search` with a number of relevant searches.
3. Because you set `enable_automatic_function_calling=True` when creating the `genai.ChatSession`, it  executed the search function and returned the list of article summaries to the model.
4. Folliwing the instructions in the prompt, the model generated a final answer based on those summaries.


## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [13]:
chat = model.start_chat()

In [14]:
result = chat.send_message(instructions.format(query=query))

Initially the model returns a FunctionCall:

In [15]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "How do deep sea creatures survive?",
      "What are the adaptations of deep sea life?",
      "Deep sea life survival",
      "Adaptations of deep sea animals",
      "Challenges of deep sea life",
      "Deep sea environment",
      "Deep sea food chain",
      "Deep sea pressure",
      "Deep sea temperature",
      "Deep sea light",
      "Deep sea organisms",
      "How do deep sea organisms survive in the absence of sunlight?"
    ]
  }
}


In [16]:
fc['name']

'wikipedia_search'

Call the function with generated arguments to get the results.

In [17]:
summaries = wikipedia_search(**fc['args'])

Searching for "How do deep sea creatures survive?"
Related search terms: ['Deep-sea community', 'Still Wakes the Deep', 'Deep sea']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Still Wakes the Deep"
Information Source: https://en.wikipedia.org/wiki/Still_Wakes_the_Deep
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Searching for "What are the adaptations of deep sea life?"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea community']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Searching for "Deep sea life survival"
Related search terms: ['Deep-sea community', 'Stranded Deep', 'Deep sea mining']
Fetching page: "Stranded Deep"
Information Source: https://en.wikipedia.org/wiki/Stranded_Deep
Stranded Deep did not match with any page id, hence skipping.
Fetching page: "Deep sea mining"
Information Source: https://en.

Now send the `FunctionResult` to the model.

In [18]:
response = chat.send_message(
    genai.protos.Content(
      parts=[genai.protos.Part(
          function_response = genai.protos.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> Deep-sea life has to survive in a very different environment from life on the surface. Here are some of the challenges they face and how they overcome them:
> 
> * **Lack of sunlight:**  Deep-sea creatures have adapted to the absence of sunlight in a few ways. Some rely on chemosynthesis, using bacteria to convert chemicals from hydrothermal vents or cold seeps into energy. Others feed on marine snow, organic matter that falls from the surface layers.  
> * **Extreme pressure:** The immense pressure at these depths would crush most surface-dwelling organisms. Deep-sea creatures have evolved to withstand this pressure by having minimal skeletons, gelatinous flesh, and no internal cavities that could collapse.
> * **Cold temperatures:** The deep sea is extremely cold, with temperatures generally remaining between 2-4 degrees Celsius. Deep-sea creatures have adapted to this by having slow metabolisms to conserve energy.
> * **Limited food:** The deep sea is a food-scarce environment. Deep-sea creatures have adapted by having large mouths and sharp teeth to capture and efficiently digest prey, and by being opportunistic feeders, consuming whatever food they can find.
> 
> Deep-sea life is incredibly diverse and has adapted to survive in one of the most challenging environments on Earth. 


## Re-ranking the search results

Helper function to embed the content:

In [21]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [19]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [22]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [23]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [24]:
print(summaries[np.argmax(sim_value)])

## Challenges of Deep Sea Life:

The provided text focuses on deep-sea exploration, not the challenges of deep-sea life itself. 
Therefore, the document does not contain information relevant to the query "Challenges of deep sea life." 


Based on:
  https://en.wikipedia.org/wiki/Deep-sea_exploration


### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-1.5-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [25]:
hypothetical_ans_model = genai.GenerativeModel('gemini-1.5-flash')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> Deep-sea life is fascinating, isn't it?  It's a whole other world down there, and it's amazing how creatures have adapted to survive in such extreme conditions.
> 
> Firstly, the pressure! It's immense, but these creatures have evolved with special [Placeholder for biological adaptation] that allow them to withstand it. 
> 
> Secondly, the darkness. Most of the deep sea is devoid of sunlight, meaning they need alternative ways to find food.  Some rely on [Placeholder for specific feeding method] while others even create their own light through [Placeholder for bioluminescence].
> 
> Thirdly, the lack of nutrients.  Food is scarce in the deep sea, so many creatures have developed [Placeholder for adaptation that helps them survive on little food]. This might include slow metabolisms, or even being able to eat just once every few months!
> 
> It's truly a marvel of evolution how life has found a way to thrive in such a harsh environment.  Imagine how many other secrets the deep sea is still hiding! 


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [26]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [27]:
sim_value = dot_product(search_res, hypothetical_ans)

In [28]:
sim_value

array([[0.86446767],
       [0.7636735 ],
       [0.84907961],
       [0.84644258],
       [0.75075276],
       [0.72567509],
       [0.78095539],
       [0.78923961],
       [0.73265915],
       [0.81705522]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [29]:
to_markdown(summaries[np.argmax(sim_value)])

> ## Relevant information about how deep-sea creatures survive:
> 
> * **High Pressure:** Deep-sea creatures have evolved to withstand the extreme pressure of the deep ocean, which increases by about one atmosphere every ten meters. They are often small, gelatinous, with minimal skeletons and lack cavities that could collapse under pressure. 
> * **Darkness:**  The deep sea is devoid of sunlight. Deep-sea creatures have adapted by:
>     * **Chemosynthesis:** Relying on bacteria that utilize chemicals from hydrothermal vents and cold seeps for energy. 
>     * **Marine Snow:** Feeding on organic matter that falls from the surface layers.
>     * **Whale Falls:**  Utilizing the large amount of organic matter provided by decaying whale carcasses.
>     * **Bioluminescence:** Some deep-sea creatures have evolved bioluminescence for communication, hunting, and camouflage.
> * **Cold Temperatures:** Deep-sea environments are extremely cold, with temperatures generally remaining between 2-4 degrees Celsius.  
> * **Limited Food:** The deep sea is a food-scarce environment. Deep-sea creatures have adapted by:
>     * **Slow Metabolism:** Having a slow metabolic rate to conserve energy.
>     * **Opportunism:** Feeding on whatever food they can find, including scavenged material and carrion.
> * **Specialized Adaptations:**  Many deep-sea creatures have unique adaptations, such as:
>     * **Large Eyes:**  For detecting bioluminescence in the darkness.
>     * **Specialized Gills:**  To extract oxygen from the low-oxygen environment.
>     * **Gelatinous Flesh:**  To reduce buoyancy and aid in survival at depth.
>     * **Reduced Skeletal Structure:**  For energy conservation.
> 
> **Important Note:** This information is focused on the adaptations that allow deep-sea creatures to survive in the harsh conditions of their environment. The text also mentions the energy sources and food webs found in the deep ocean, which are essential for understanding how these creatures thrive. 
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_community

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.